### scrapy 
#### 파이썬 코드로 웹페이지의 데이터를 수집하는 프레임워크
- 비동기 방식으로 데이터를 수집해 속도가 빠르다

In [1]:
import scrapy, requests
from scrapy.http import TextResponse

In [ ]:
# gmarket 베스트 200 상품 데이터 수집 

In [ ]:
# 1. scrapy project 생성 (project name : gmarket)

In [2]:
!scrapy startproject gmarket

New Scrapy project 'gmarket', using template directory 'c:\users\user\anaconda3\lib\site-packages\scrapy\templates\project', created in:
    C:\Users\User\Downloads\AIVLE SCHOOL\웹크롤링\day3\gmarket

You can start your first spider with:
    cd gmarket
    scrapy genspider example example.com


In [3]:
!tree gmarket /f

폴더 PATH의 목록입니다.
볼륨 일련 번호가 00000086 7C6C:FC35입니다.
C:\USERS\USER\DOWNLOADS\AIVLE SCHOOL\웹크롤링\DAY3\GMARKET
│  scrapy.cfg
│  
└─gmarket
    │  items.py
    │  middlewares.py
    │  pipelines.py
    │  settings.py
    │  __init__.py
    │  
    └─spiders
            __init__.py
            


In [ ]:
# item.py : 수집할 데이터의 구조 정의
# middlewares.py : 데이터를 수집할 때 headers 정보와 같은 내용을 설정 
# piplines.py : 데이터 수집 후 코드 실행을 정의
# setting.py : 크롤링 설정 : 크롤링 시간 텀, robots.txt 규칙 지킬지 말지
# spiders : 디렉토리 : 크롤링 절차 정의 

In [ ]:
# 2. xpath 찾기 : link, 상세 페이지의 데이터

In [ ]:
url = 'http://corners.gmarket.co.kr/Bestsellers'

In [ ]:
# link 데이터 수집 

In [4]:
request = requests.get('http://corners.gmarket.co.kr/Bestsellers')
response = TextResponse(request.url, body=request.text, encoding="utf-8")

<200 http://corners.gmarket.co.kr/Bestsellers>

In [6]:
links = response.xpath('//*[@id="gBestWrap"]/div/div[3]/div/ul/li/a/@href').extract()
# 텍스트쪽 xpath # 모든 거 가져올 거니까 li[1]에서 [1] 지운다! 
# @href : 링크
# .extract() : 그거만 추출하기 
links[:2]

['http://item.gmarket.co.kr/Item?goodscode=2519604129&ver=637953125702971589',
 'http://item.gmarket.co.kr/Item?goodscode=2520466046&ver=637953125702971589']

In [ ]:
# 상세 페이지 데이터 수집

In [11]:
link = links[0] # 이 url에 있는 상세 페이지 데이터 가져올거임
request = requests.get(link)
response = TextResponse(request.url, body=request.text, encoding="utf-8")

In [12]:
title = response.xpath('//*[@id="itemcase_basic"]/div/h1/text()')[0].extract()
price = response.xpath('//*[@id="itemcase_basic"]/div/p/span/strong/text()')[0].extract()
title, price

('브라운 면도기 세정액 CCR5+1 ', '25,030')

In [ ]:
# 3. item.py : 코드 작성 : model(데이터의 구조 : 수집할 데이터의 컬럼을 정의)

In [13]:
%%writefile gmarket/gmarket/items.py
import scrapy

class GmarketItem(scrapy.Item):
    title = scrapy.Field()
    price = scrapy.Field()
    link = scrapy.Field()

Overwriting gmarket/gmarket/items.py


In [ ]:
# 4. spider.py : 코드 작성 : 크롤링 절차 정의 

In [19]:
%%writefile gmarket/gmarket/spiders/spider.py
import scrapy
from gmarket.items import GmarketItem

class GMSpider(scrapy.Spider):
    name = "GMB"
    allow_domain = ["gmarket.co.kr"] 
    # 크롤링 할 url 도메인 설정: 네이버나 쿠팡 url은 수집 안하겠다 지마켓 url만 수집하겠다
    start_urls = ["http://corners.gmarket.co.kr/Bestsellers"]
    # 최초의 request response하는 url
    
    def parse(self, response):
        links = response.xpath('//*[@id="gBestWrap"]/div/div[3]/div/ul/li/a/@href').extract()
        for link in links[:20]: # 상품 200개 중에 20개만... 오래 걸리니까~
            yield scrapy.Request(link, callback=self.parse_content)
    
    def parse_content(self, response):
        item = GmarketItem()
        item["title"] = response.xpath('//*[@id="itemcase_basic"]/div[1]/h1/text()')[0].extract()
        item["price"] = response.xpath('//*[@id="itemcase_basic"]/div[1]/p/span/strong/text()')[0].extract()
        item["link"] = response.url
        yield item

Writing gmarket/gmarket/spiders/spider.py


In [ ]:
# 5. scrapy project 실행 
# gmarket 디렉토리
# scrapy crwal GMB -o items.csv : 터미널에서 실행 

In [ ]:
# 터미널 창에서 어떻게 하는지 못봤다.. 복습 필요 ㅠ

In [21]:
%pwd

'C:\\Users\\User\\Downloads\\AIVLE SCHOOL\\웹크롤링\\day3'

In [ ]:
pd.read_csv("gmarket/items.csv").tail(2)